In [38]:
import numpy as np
from gradvi.optimize import root_find

import matplotlib.pyplot as plt
from pymir import mpl_stylesheet
from pymir import mpl_utils
mpl_stylesheet.banskt_presentation(splinecolor = 'black')

In [39]:
_epsilon = np.sqrt(np.finfo(float).eps)

def cubefun(x, b, a):
    x2 = np.square(x)
    f = a * x2 * x - b
    return f

def cubefun_grad(x, b, a):
    x2 = np.square(x)
    f  = a * x2 * x - b
    df = 3.0 * a * x2
    return f, df

def cubefun_solve(b, a):
    xr = np.cbrt(b / a)
    return xr


def cubefun_bracket(b, a):
    xr = cubefun_solve(b, a)
    n  = xr.shape[0]
    xlo = xr - np.abs(np.random.normal(1, 0.2, size = n))
    xup = xr + np.abs(np.random.normal(1, 0.2, size = n))
    return [xlo, xup]

In [42]:
np.random.seed(100)
n = 10
a = np.random.normal(10, 1, size = n)
b = np.random.normal(2, 1, size = n)
x0 = np.ones(n)
xtrue = cubefun_solve(b, a)

In [43]:
xtrue

array([0.57174647, 0.61749248, 0.50264895, 0.66113334, 0.62435703,
       0.56492272, 0.52377644, 0.69745842, 0.54198092, 0.44135906])

In [44]:
cubefun(xtrue, b, a)

array([-4.44089210e-16, -8.88178420e-16, -1.33226763e-15,  0.00000000e+00,
       -1.77635684e-15, -1.55431223e-15, -4.44089210e-16, -1.77635684e-15,
       -6.66133815e-16, -1.11022302e-16])

In [45]:
cubefun_grad(xtrue, b, a)

(array([-4.44089210e-16, -8.88178420e-16, -1.33226763e-15,  0.00000000e+00,
        -1.77635684e-15, -1.55431223e-15, -4.44089210e-16, -1.77635684e-15,
        -6.66133815e-16, -1.11022302e-16]),
 array([ 8.09085718, 11.83089787,  8.45364314, 12.78190148, 12.8422714 ,
        10.06645037,  8.41228919, 13.03188523,  8.64531012,  5.99295567]))

In [46]:
cubefun_grad(np.zeros(n), b, a)

(array([-1.54197301, -2.43516349, -1.41640495, -2.81684707, -2.67272081,
        -1.89558886, -1.46871962, -3.02973269, -1.56186438, -0.88168175]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [47]:
cubefun_bracket(b, a)

[array([-0.75204986, -0.69082856, -0.44697522, -0.17037951, -0.41254671,
        -0.62249372, -0.62242363, -0.57485281, -0.39277147, -0.56977615]),
 array([1.61622639, 1.32884908, 1.35137849, 1.82442414, 1.77444598,
        1.47373334, 1.76170089, 1.35933505, 1.27070111, 1.19487215])]

In [51]:
from gradvi.optimize import bracket
#xlo, xup, nfev = bracket.bracket_postmean(cubefun, b, args = (a,))
xlo, xup = cubefun_bracket(b, a)

In [52]:
xlo

array([-0.3193657 , -0.24887317, -0.49881396, -0.21627891, -0.63559259,
       -0.08845815, -0.27956154, -0.37404313, -0.13530338, -0.85278372])

In [53]:
xup

array([1.33414295, 1.50754324, 1.31463972, 1.49554687, 1.64612972,
       1.66648464, 1.35133097, 1.94735237, 1.52605867, 1.26341276])

In [54]:
cubefun(xlo, b, a)

array([-1.81071226, -2.5945925 , -2.800637  , -2.91546108, -5.49234382,
       -1.9028665 , -1.69204255, -3.49705273, -1.58616499, -7.24161305])

In [55]:
cubefun(xup, b, a)

array([18.04978433, 33.00065257, 23.9239879 , 29.78905171, 46.31043348,
       46.76540329, 23.75366895, 62.91543538, 33.30436789, 19.79931671])

In [58]:
root_find.vec_root(cubefun, np.zeros(n), args = (b, np.zeros(n)), method = 'trisection',
                   bracket = [b - 1., b + 1.])

ValueError: Trisection - function must have opposite signs at the boundaries.

In [59]:
cubefun(b - 1.0, b, np.zeros(n))

array([-1.54197301, -2.43516349, -1.41640495, -2.81684707, -2.67272081,
       -1.89558886, -1.46871962, -3.02973269, -1.56186438, -0.88168175])

In [61]:
cubefun(b + 1.0, b, np.zeros(n))

array([-1.54197301, -2.43516349, -1.41640495, -2.81684707, -2.67272081,
       -1.89558886, -1.46871962, -3.02973269, -1.56186438, -0.88168175])

In [37]:
def cubefun_grad_fssi(x, a):
    x2 = np.square(x)
    f  = a * x2 * x
    df = 3.0 * a * x2
    return f, df

#root_find.vec_root(cubefun_grad_fssi, np.zeros(n), args = (a,), fx = b,
#                   method = 'fssi-linear', jac = True, bounds = [-3, 3])

try:
    root_find.root_fssi(cubefun_grad_fssi, b, args = (0.0,), bounds = [0.33, 3.1], full_output=True)
except ArithmeticError as err:
    print("Could not find solution using FSSI.")
    root_find.vec_root(cubefun, np.zeros(n), args = (b, np.zeros(n),), method = 'trisection',
                       bracket = [xlo - 3, xup])

Could not find solution using FSSI.


ValueError: Trisection - function must have opposite signs at the boundaries.

In [28]:
a

array([ 8.25023453, 10.3426804 , 11.1530358 ,  9.74756396, 10.98132079,
       10.51421884, 10.22117967,  8.92995667,  9.81050417, 10.25500144])

In [12]:
model, mu, beta = fit_ash_trendfiltering_gradvi(X, y, objtype,
                    degree = degree, ncomp = ncomp, sparsity = sparsity, skbase = skbase,
                    binit = binit, s2init = s2init)